## Simple Reinforcement Learning
### CartPole - Tensorflow

09/02/18 - This is a policy-gradient reinforcement agent.
Honestly still don't understand what I'm doing.
I think the final outputs are the score that we are hitting in the game CartPole.

Acknowledgment: https://github.com/awjuliani/DeepRL-Agents/blob/master/Vanilla-Policy.ipynb

In [3]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plt
%matplotlib inline

try:
    xrange = xrange
except:
    xrange = range

In [4]:
env = gym.make('CartPole-v0')

[2018-02-09 20:23:43,817] Making new env: CartPole-v0


In [16]:
gamma = 0.99

def discount_rewards(r):
    """ take 1d float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [17]:
class agent():
    def __init__(self, lr, s_size, a_size, h_size):
        # Some feedforward stuff. The agent takes a state and produces an action?
        self.state_in = tf.placeholder(shape=[None, s_size],dtype=tf.float32)
        hidden = slim.fully_connected(self.state_in, h_size,biases_initializer=None, activation_fn=tf.nn.relu)
        self.output = slim.fully_connected(hidden, a_size, activation_fn=tf.nn.softmax, biases_initializer=None)
        self.chosen_action = tf.argmax(self.output, 1)
        
        # Establish training proceedure. We feed the reward and chosen action into the network
        # to compute the loss, and use it to update the network
        self.reward_holder = tf.placeholder(shape=[None], dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None], dtype=tf.int32)
        
        self.indexes = (tf.range(0, tf.shape(self.output)[0]) 
                              * tf.shape(self.output)[1] 
                              + self.action_holder)
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), 
                                             self.indexes)
        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs)
                                    *self.reward_holder)
        
        tvars = tf.trainable_variables()
        self.gradient_holders = []
        for idx,var in enumerate(tvars):
            placeholder = tf.placeholder(tf.float32, name=str(idx)+'_holder')
            self.gradient_holders.append(placeholder)
            
        self.gradients = tf.gradients(self.loss, tvars)
        
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders,tvars))

## Traning the Agent

In [18]:
tf.reset_default_graph() # Clear the Tensorflow graph
myAgent = agent(lr=1e-2, s_size=4, a_size=2,h_size=8) # load the agent

total_episodes = 5000 # Set total number of episodes to train agent on
max_ep = 999
update_frequency = 5

init = tf.global_variables_initializer()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    total_reward = []
    total_length= []
    
    gradBuffer = sess.run(tf.trainable_variables())
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
        
    while i < total_episodes:
        s = env.reset()
        running_reward = 0
        ep_history = []
        for j in range (max_ep):
            #Probablilistically pick an action given our network outputs
            a_dist = sess.run(myAgent.output, feed_dict={myAgent.state_in:[s]})
            a = np.random.choice(a_dist[0], p=a_dist[0])
            a = np.argmax(a_dist == a)
            
            s1,r,d,_ = env.step(a) # Get our reward for taking an action given a bandit
            ep_history.append([s,a,r,s1])
            s = s1
            running_reward += r
            if d == True:
                # update the network
                ep_history = np.array(ep_history)
                ep_history[:,2] = discount_rewards(ep_history[:,2])
                feed_dict={myAgent.reward_holder:ep_history[:,2],
                          myAgent.action_holder:ep_history[:,1],
                          myAgent.state_in:np.vstack(ep_history[:,0])}
                grads = sess.run(myAgent.gradients, feed_dict=feed_dict)
                for idx, grad in enumerate(grads):
                    gradBuffer[idx] += grad
                    
                if i % update_frequency == 0 and i != 0:
                    feed_dict = dictionary = dict(zip(myAgent.gradient_holders, gradBuffer))
                    _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
                    for ix,grad in enumerate(gradBuffer):
                        gradBuffer[ix] = grad * 0
                        
                total_reward.append(running_reward)
                total_length.append(j)
                break
                
            # update our running tally of scores
        if i % 100 == 0:
            print(np.mean(total_reward[-100:]))
        i += 1

/home/master/anaconda3/envs/my_env35/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


19.0
25.95
35.15
45.71
67.39
97.72
146.94
170.04
181.47
187.28
185.26
187.7
184.33
172.91
170.12
190.42
192.38
196.53
193.91
195.66
196.41
197.17
198.6
196.54
195.68
193.77
197.86
199.87
199.51
198.19
199.81
200.0
199.97
200.0
199.55
199.63
200.0
199.01
198.69
198.35
194.89
194.66
195.72
196.6
198.67
197.97
198.08
197.6
189.14
194.84
